In [1]:
import sys
sys.path.insert(0, '../')
from gw_tools import data_prep
from gw_tools.gw_cnn import gw_cnn
from gw_tools.gw_LSTM import gw_LSTM
from gw_tools import model_params

import numpy as np
import pandas as pd
import pickle

from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

2023-11-25 10:42:55.707055: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-25 10:42:55.740766: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-25 10:42:55.740789: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-25 10:42:55.740821: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-25 10:42:55.747296: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-25 10:42:55.747868: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
prediction_path = '../data/prediction_pickles/'
score_path = '../data/prediction_pickles/'
future_pred_path = '../data/prediction_pickles/'
future_score_path = '../data/prediction_pickles/'


score_summary = pd.DataFrame({'Model':['Baseline', 'Linear Regression', 'CNN', 'LSTM'],'RMSE':[0,0,0,0], 'MAE':[0,0,0,0]})
score_summary_future = pd.DataFrame({'Model':['Baseline', 'Linear Regression', 'CNN', 'LSTM'],'RMSE':[0,0,0,0], 'MAE':[0,0,0,0]})

In [3]:
## Define a scaler
scaler_cnn =  StandardScaler(copy=True)
scaler_lstm =  StandardScaler(copy=True)


## note: parameters are stored in model_params, imported above
for well, params, LSTMparams in zip(model_params.wells, 
                                    model_params.CNN_well_params, 
                                    model_params.LSTM_well_params):
    ## Prepare the data for training
    df = data_prep.load_data(well)
    well_dates = df['date'].copy()
    df = data_prep.select_features(df)
    df = data_prep.add_toy_signal(df)
    
    ## LSTM specific prep (requires a warmup, target as a feature)
    # we also need the window size later for the future predictions
    window = LSTMparams['model__WINDOW_SIZE']
    X_tr_L, X_te_L, well_tr_mean, well_tr_std = data_prep.LSTM_data_prep(df,window)
    # Set the mean and standard deviation of the training set as parameters
    LSTMparams['model__tmean'] = well_tr_mean
    LSTMparams['model__tsd'] = well_tr_std
    
    
    ## CNN data
    X_train, X_holdout, y_train, y_holdout, dt_train, dt_holdout = data_prep.prep_data_for_training(df)
    
    
    ## Get average for baseline prediction
    train_mean = y_train.mean()
    
    ## Train the linear regression model
    model_lr = LinearRegression(copy_X=True)
    model_lr.fit(X_train, y_train)
    
    ## Train the CNN
    model_cnn = gw_cnn()
    pipe_cnn = Pipeline([('scaler', scaler_cnn), ('model', model_cnn)])
    pipe_cnn.set_params(**params)
    pipe_cnn.fit(X_train, y_train)

    ## Train the LSTM
    model_LSTM = gw_LSTM()
    pipe_LSTM = Pipeline([('scaler', scaler_lstm), ('model', model_LSTM)])
    pipe_LSTM.set_params(**LSTMparams)
    pipe_LSTM.fit(X_tr_L, y_train)

    
    ## Prepare future data for predictions and scoring
    features = data_prep.load_data('FEATS')
    features = data_prep.select_features(features, no_target=True)
    features = data_prep.add_toy_signal(features)
    future_dates = features.loc[features.date > data_prep.get_end_date(well)]['date'].copy()
    # CNN specific future data
    X_future = features.loc[features.date > data_prep.get_end_date(well)].drop('date', axis=1).copy()
    all_dates = pd.concat([well_dates, future_dates])
    
    ## Keep track of the length of predictions
    len_pred_test = len(y_holdout)
    len_pred_future = all_dates.shape[0]-len_pred_test
    
    ## Make baseline predictions
    pred_avg_train = np.ones(len_pred_test)*train_mean
    pred_avg_future = np.ones(len_pred_future)*train_mean
    pred_avg_full = np.append(pred_avg_train, pred_avg_future)
    
    ## Make linear regression predictions
    pred_lin_train = model_lr.predict(X_train)
    pred_lin_test = model_lr.predict(X_holdout)
    pred_lin_future = model_lr.predict(X_future.values)
    pred_lin_full = np.append(pred_lin_train, 
                              np.append(pred_lin_test, pred_lin_future))
    
    ## Make CNN predictions
    pred_cnn_train = pipe_cnn.predict(X_train)
    pred_cnn_test = pipe_cnn.predict(X_holdout)
    pred_cnn_future = pipe_cnn.predict(X_future.values)
    pred_cnn_full = np.append(pred_cnn_train, 
                              np.append(pred_cnn_test, pred_cnn_future))

    ## Make LSTM prediction
    # the LSTM requires a warmup set, so the first window=WINDOW_SIZE predictions for the training
    # set will just be the actual values
    ytr_warmup = y_train[:window]
    pred_LSTM_train = np.concatenate((ytr_warmup, pipe_LSTM.predict(X_tr_L)))
    pred_LSTM_test = pipe_LSTM.predict(X_te_L)
    # LSTM specific future data (requires warmup set built from the predictions on the test set
    # or from the actual values, tbd)
    # if from actual values, replace pred_LSTM_test with y_holdout in the call of LSTM_future
    X_fut_L = data_prep.LSTM_future(X_future, pred_LSTM_test, X_te_L, window)
    pred_LSTM_future = pipe_LSTM.predict(X_fut_L)
    pred_LSTM_full = np.append(pred_LSTM_train, 
                              np.append(pred_LSTM_test, pred_LSTM_future))
    
    ## Gather the predictions and actual data into a single dataframe
    well_data = data_prep.load_data(well)
    full_predict = well_data[['date', 'avg_well_depth']].merge(all_dates, on='date', how='outer')
    full_predict.rename(columns={'avg_well_depth':'Actual'}, inplace=True)
    full_predict['Baseline'] = pred_avg_full
    full_predict['Linear Reg'] = pred_lin_full
    full_predict['CNN'] = pred_cnn_full
    full_predict['LSTM'] = pred_LSTM_full
    
    ## Isolate the prediction on the holdout set for scoring
    test_predict = full_predict.loc[full_predict['date']<=data_prep.get_end_date(well)][-365:].copy()
    
    ## Compute scores on the holdout set
    RMSE_baseline = np.sqrt(np.mean((test_predict.Actual - test_predict.Baseline)**2))
    MAE_baseline = np.mean(np.abs(test_predict.Actual - test_predict.Baseline))
    
    RMSE_lin_reg = np.sqrt(np.mean((test_predict.Actual - test_predict['Linear Reg'])**2))
    MAE_lin_reg = np.mean(np.abs(test_predict.Actual - test_predict['Linear Reg']))
    
    RMSE_cnn = np.sqrt(np.mean((test_predict.Actual - test_predict.CNN)**2))
    MAE_cnn = np.mean(np.abs(test_predict.Actual - test_predict.CNN))

    RMSE_lstm = np.sqrt(np.mean((test_predict.Actual - test_predict.LSTM)**2))
    MAE_lstm = np.mean(np.abs(test_predict.Actual - test_predict.LSTM))
    
    ## Update the scores in the dataframe
    score_summary.RMSE = [RMSE_baseline, RMSE_lin_reg, RMSE_cnn, RMSE_lstm]
    score_summary.MAE = [MAE_baseline, MAE_lin_reg, MAE_cnn, MAE_lstm]
    
    ## Save the results for this well in a dataframe
    prediction_filename = 'model_predictions_'+well+'.pkl'
    score_filename = 'model_scores_'+well+'.pkl'

    full_predict.to_pickle(prediction_path+prediction_filename)
    score_summary.to_pickle(score_path+score_filename)

2023-11-25 10:42:57.382906: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-25 10:42:57.383413: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/100
124/124 [==============================] - 1s 4ms/step - loss: 824.5236
Epoch 2/100
124/124 [==============================] - 0s 4ms/step - loss: 86.6375
Epoch 3/100
124/124 [==============================] - 0s 4ms/step - loss: 16.5056
Epoch 4/100
124/124 [==============================] - 0s 4ms/step - loss: 3.1008
Epoch 5/100
124/124 [==============================] - 0s 4ms/step - loss: 1.9910
Epoch 6/100
124/124 [==============================] - 0s 4ms/step - loss: 1.2553
Epoch 7/100
124/124 [==============================] - 0s 4ms/step - loss: 0.8167
Epoch 8/100
124/124 [==============================] - 0s 4ms/step - loss: 0.6967
Epoch 9/100
124/124 [==============================] - 0s 4ms/step - loss: 0.5722
Epoch 10/100
124/124 [==============================] - 0s 4ms/step - loss: 0.4844
Epoch 11/100
124/124 [==============================] - 0s 4ms/step - loss: 0.4578
Epoch 12/100
124/124 [==============================] - 0s 4ms/step - loss: 0.5611
Epoch 13/

/home/meredith/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/meredith/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/meredith/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/meredith/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isi

Epoch 1/30
123/123 [==============================] - 2s 6ms/step - loss: 2935.7791 - root_mean_squared_error: 54.1828
Epoch 2/30
123/123 [==============================] - 1s 6ms/step - loss: 258.5028 - root_mean_squared_error: 16.0780
Epoch 3/30
123/123 [==============================] - 1s 6ms/step - loss: 10.2460 - root_mean_squared_error: 3.2009
Epoch 4/30
123/123 [==============================] - 1s 6ms/step - loss: 8.6822 - root_mean_squared_error: 2.9466
Epoch 5/30
123/123 [==============================] - 1s 6ms/step - loss: 4.4973 - root_mean_squared_error: 2.1207
Epoch 6/30
123/123 [==============================] - 1s 6ms/step - loss: 3.6416 - root_mean_squared_error: 1.9083
Epoch 7/30
123/123 [==============================] - 1s 6ms/step - loss: 1.7424 - root_mean_squared_error: 1.3200
Epoch 8/30
123/123 [==============================] - 1s 6ms/step - loss: 0.3458 - root_mean_squared_error: 0.5881
Epoch 9/30
123/123 [==============================] - 1s 6ms/step - loss

/home/meredith/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/meredith/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/meredith/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


...

1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 12ms/step


1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 12ms/step


In [6]:
pred_AEK201 = pd.read_pickle('../data/prediction_pickles/model_predictions_AEK201.pkl')
pred_AFL259 = pd.read_pickle('../data/prediction_pickles/model_predictions_AFL259.pkl')
pred_APK309 = pd.read_pickle('../data/prediction_pickles/model_predictions_APK309.pkl')
pred_APK310 = pd.read_pickle('../data/prediction_pickles/model_predictions_APK310.pkl')

future_AEK201 = pd.read_csv('../data/raw_data/EIM-data-AEK201/EIMDiscreteResults_2023Oct22_39.csv')
future_AEK201 = future_AEK201[['Result_Value','Field_Collection_End_Date']].copy()
future_AEK201['date'] = pd.to_datetime(future_AEK201['Field_Collection_End_Date'])
future_AEK201 = future_AEK201.drop('Field_Collection_End_Date', axis=1).copy()
future_AEK201 = future_AEK201[['date','Result_Value']].sort_values('date').reset_index(drop=True)
future_AEK201 = pred_AEK201[['date', 'Baseline', 'Linear Reg', 'CNN', 'LSTM']].merge(future_AEK201, on='date').copy()
future_AEK201 = future_AEK201.rename(columns={'Result_Value': 'actual_depth'}).copy()
future_AEK201 = future_AEK201.loc[future_AEK201.date > data_prep.get_end_date('AEK201')]

future_AFL259 = pd.read_csv('../data/raw_data/EIM-data-AFL259/GroundwaterLevelsDiscreteResults_2023Nov12_24.csv')
future_AFL259 = future_AFL259[['Water_Level_Value','Field_Collection_Date']].copy()
future_AFL259['date'] = pd.to_datetime(future_AFL259['Field_Collection_Date'])
future_AFL259 = future_AFL259.drop('Field_Collection_Date', axis=1).copy()
future_AFL259 = future_AFL259[['date','Water_Level_Value']].sort_values('date').reset_index(drop=True)
future_AFL259 = pred_AFL259[['date', 'Baseline', 'Linear Reg', 'CNN', 'LSTM']].merge(future_AFL259, on='date').copy()
future_AFL259 = future_AFL259.rename(columns={'Water_Level_Value': 'actual_depth'}).copy()
future_AFL259 = future_AFL259.loc[future_AFL259.date > data_prep.get_end_date('AFL259')]

future_APK309 = pd.read_csv('../data/raw_data/EIM-data-APK309/GroundwaterLevelsDiscreteResults_2023Oct19_19.csv')
future_APK309 = future_APK309[['Water_Level_Value','Field_Collection_Date']].copy()
future_APK309['date'] = pd.to_datetime(future_APK309['Field_Collection_Date'])
future_APK309 = future_APK309.drop('Field_Collection_Date', axis=1).copy()
future_APK309 = future_APK309[['date','Water_Level_Value']].sort_values('date').reset_index(drop=True)
future_APK309 = pred_APK309[['date', 'Baseline', 'Linear Reg', 'CNN', 'LSTM']].merge(future_APK309, on='date').copy()
future_APK309 = future_APK309.rename(columns={'Water_Level_Value': 'actual_depth'}).copy()
future_APK309 = future_APK309.loc[future_APK309.date > data_prep.get_end_date('APK309')]

future_APK310 = pd.read_csv('../data/raw_data/EIM-data-APK310/GroundwaterLevelsDiscreteResults_2023Nov02_19.csv')
future_APK310 = future_APK310[['Water_Level_Value','Field_Collection_Date']].copy()
future_APK310['date'] = pd.to_datetime(future_APK310['Field_Collection_Date'])
future_APK310 = future_APK310.drop('Field_Collection_Date', axis=1).copy()
future_APK310 = future_APK310[['date','Water_Level_Value']].sort_values('date').reset_index(drop=True)
future_APK310 = pred_APK310[['date', 'Baseline', 'Linear Reg', 'CNN', 'LSTM']].merge(future_APK310, on='date').copy()
future_APK310 = future_APK310.rename(columns={'Water_Level_Value': 'actual_depth'}).copy()
future_APK310 = future_APK310.loc[future_APK310.date > data_prep.get_end_date('APK310')]

future_AEK201.to_pickle(future_pred_path+'future_data_compare_AEK201.pkl')
future_AFL259.to_pickle(future_pred_path+'future_data_compare_AFL259.pkl')
future_APK309.to_pickle(future_pred_path+'future_data_compare_APK309.pkl')
future_APK310.to_pickle(future_pred_path+'future_data_compare_APK310.pkl')


future_preds = [future_AEK201, future_AFL259, future_APK309, future_APK310]
for well, pred in zip(model_params.wells, future_preds):
    ## Compute scores on the holdout set
    RMSE_baseline = np.sqrt(np.mean((pred.actual_depth - pred.Baseline)**2))
    MAE_baseline = np.mean(np.abs(pred.actual_depth - pred.Baseline))
    
    RMSE_lin_reg = np.sqrt(np.mean((pred.actual_depth - pred['Linear Reg'])**2))
    MAE_lin_reg = np.mean(np.abs(pred.actual_depth - pred['Linear Reg']))
    
    RMSE_cnn = np.sqrt(np.mean((pred.actual_depth - pred.CNN)**2))
    MAE_cnn = np.mean(np.abs(pred.actual_depth - pred.CNN))

    RMSE_lstm = np.sqrt(np.mean((pred.actual_depth - pred.LSTM)**2))
    MAE_lstm = np.mean(np.abs(pred.actual_depth - pred.LSTM))
    
    ## Update the scores in the dataframe
    score_summary.RMSE = [RMSE_baseline, RMSE_lin_reg, RMSE_cnn, RMSE_lstm]
    score_summary.MAE = [MAE_baseline, MAE_lin_reg, MAE_cnn, MAE_lstm]
    
    ## Save the results for this well in a dataframe
    score_filename = 'model_future_scores_'+well+'.pkl'
    score_summary.to_pickle(score_path+score_filename)

In [7]:
pd.read_pickle('../data/prediction_pickles/model_scores_AEK201.pkl')

,Model,RMSE,MAE
0,Baseline,4.390765,3.285349
1,Linear Regression,0.854502,0.663539
2,CNN,1.653215,1.346199
3,LSTM,0.347969,0.308972


In [8]:
pd.read_pickle('../data/prediction_pickles/model_scores_AFL259.pkl')

,Model,RMSE,MAE
0,Baseline,6.896880,5.298502
1,Linear Regression,4.811088,4.109074
2,CNN,5.483122,4.769778
3,LSTM,2.302446,2.044797


In [9]:
pd.read_pickle('../data/prediction_pickles/model_scores_APK309.pkl')

,Model,RMSE,MAE
0,Baseline,3.205925,2.371962
1,Linear Regression,1.731512,1.586595
2,CNN,2.007611,1.820090
3,LSTM,1.118477,0.914378


In [10]:
pd.read_pickle('../data/prediction_pickles/model_scores_APK310.pkl')

,Model,RMSE,MAE
0,Baseline,5.060397,3.689585
1,Linear Regression,2.324117,1.571088
2,CNN,2.618668,1.842734
3,LSTM,1.190798,0.998867


In [11]:
pd.read_pickle('../data/prediction_pickles/model_future_scores_AEK201.pkl')

,Model,RMSE,MAE
0,Baseline,5.306228,4.610481
1,Linear Regression,1.278526,1.268504
2,CNN,1.135177,1.075799
3,LSTM,0.925442,0.775610


In [12]:
pd.read_pickle('../data/prediction_pickles/model_future_scores_AFL259.pkl')

,Model,RMSE,MAE
0,Baseline,6.044469,5.119710
1,Linear Regression,3.992602,2.988603
2,CNN,3.874197,3.072574
3,LSTM,2.785905,2.390334


In [13]:
pd.read_pickle('../data/prediction_pickles/model_future_scores_APK309.pkl')

,Model,RMSE,MAE
0,Baseline,3.332982,2.924752
1,Linear Regression,1.061081,0.994386
2,CNN,0.930967,0.789103
3,LSTM,0.560578,0.456298


In [14]:
pd.read_pickle('../data/prediction_pickles/model_future_scores_APK310.pkl')

,Model,RMSE,MAE
0,Baseline,6.365648,5.463669
1,Linear Regression,1.696729,1.440611
2,CNN,1.122035,1.022336
3,LSTM,1.201338,0.851242
